# 前処理  
訓練データには多くの欠損値、データ型がバラバラなど、うまく学習させることができない要因が多い。  
欠損値の補填、新たな特徴の抽出、データ型の統一を行う。

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
# 訓練データ
train = pd.read_csv('./dataset/train.csv')

## 特徴量
PassengerId 乗客ID, Survived 生存, Pclass 階級, Name 名前, Sex 性別, Age 年齢, SibSp 夫婦・兄弟, Parch 親・子  
Ticket　チケット番号, Fare 運賃, Cabin 客室番号, Embarked 寄港

In [3]:
# 最初の訓練データ
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# それぞれの欠損値の数
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

欠損値は、年齢に177個, 客室番号に687個, 寄港地に2個あることがわかる。

## 特徴の追加

### 敬称(Title)の追加

全ての名前(Name)には、敬称(Mr., Mrs., Miss. …)が付いていることがわかる。  
敬称は、その人の肩書きや職業の社会的地位、性別、大体の年齢など、その人がどういう人なのかを簡単に表したもの。  
名前は全員が異なるので使用できないが、敬称には何かしらパターンがあるのではないかと考える。

In [5]:
# 名前(Name)から敬称(Title)を抽出
train['Title'] = train['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [6]:
Counter(train['Title'])

Counter({'Capt': 1,
         'Col': 2,
         'Countess': 1,
         'Don': 1,
         'Dr': 7,
         'Jonkheer': 1,
         'Lady': 1,
         'Major': 2,
         'Master': 40,
         'Miss': 182,
         'Mlle': 2,
         'Mme': 1,
         'Mr': 517,
         'Mrs': 125,
         'Ms': 1,
         'Rev': 6,
         'Sir': 1})

敬称には  
Cap:船長, Col:少佐・大佐, Countess:女伯爵, Don:貴人・高位聖職者, Dr:医者・博士号, Jonkheer:男爵, Lady:貴婦人 ,Major:少佐, Master:少年, Miss:未婚女性, Mlle:未婚女性, Mme:既婚女性, Mr:成人男性, Mrs:既婚女性, Ms:女性, Rev:牧師, Sir:騎士  
と意外と種類の多いことがわかる。  　

敬称は社会的地位や年齢を表していることから、年齢の補填にいいのではないか。  
同じ敬称の平均値を補填してみる。

In [7]:
# 年齢に欠損のある敬称
Counter(train[train['Age'].isnull()].Title)

Counter({'Dr': 1, 'Master': 4, 'Miss': 36, 'Mr': 119, 'Mrs': 17})

運のいいことに年齢に欠損値がある人は、同じ敬称の人がいるものであった。

In [8]:
# 欠損値のある敬称の平均値
Dr_age = train.query('Title == "Dr"').Age.mean()
Master_age = train.query('Title == "Master"').Age.mean()
Miss_age = train.query('Title == "Miss"').Age.mean()
Mr_age = train.query('Title == "Mr"').Age.mean()
Mrs_age = train.query('Title == "Mrs"').Age.mean()
Ms_age = train.query('Title == "Ms"').Age.mean()

In [9]:
# 同敬称の年齢の平均値を補填
train.loc[(train["Title"].values == "Dr") & (train["Age"].isnull()), "Age"] = Dr_age
train.loc[(train["Title"].values == "Master") & (train["Age"].isnull()), "Age"] = Master_age
train.loc[(train["Title"].values == "Miss") & (train["Age"].isnull()), "Age"] = Miss_age
train.loc[(train["Title"].values == "Mr") & (train["Age"].isnull()), "Age"] = Mr_age
train.loc[(train["Title"].values == "Mrs") & (train["Age"].isnull()), "Age"] = Mrs_age

敬称には、MrやMrsなど一般的なものから、MlleやJonkheerなどヨーロッパの貴族の敬称がある。  
名前よりも種類は減ったものの、一人や数人しかいないものもあるので、近い敬称とそれ以外にする。  
残す敬称は、少年(Master), 未婚女性(Miss), 既婚女性(Mrs), 男性(Mr), その他(Others)

In [10]:
# 近い敬称をまとめる
train['Title'] = train['Title'].replace(['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Sir', 'Jonkheer', 'Lady', 'Capt', 'Don'], 'Others')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')
train['Title'] = train['Title'].replace('Mlle', 'Miss')

### グループ人数  
救命ボートに乗る際、複数人のグループで乗るか、一人で乗るかによって、生存に影響が出るのではないかと考えた。  
例えば、一人や少人数であれば満員の救命ボートに潜り込める。一方で複数人グループで誰かが乗れないと、一緒に降りるといったことがあったかもしれない。  
  
元のデータには、夫婦・兄弟、親・子の特徴があったので、これよりグループの人数を求める。

In [11]:
# 特徴「家族の人数」を追加
train['GroupSize'] = train['SibSp'] + train['Parch'] + 1

In [12]:
# 補填と特徴を追加した訓練データ
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,GroupSize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


## 必要ない特徴の除外、欠損値の補填、数値化  
乗客ID, 名前, 夫婦・兄弟の数, 親・子の数, チケット番号, 客室番号を除外  
寄港地の欠損値はSを補填  
性別・寄港地・敬称をそれぞれ数値化  

In [13]:
train = train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)
train = train.fillna({'Embarked' :'S'})
train = train.replace({
    'male' : 0, 'female' : 1, 
    'S' : 0, 'C' : 1, 'Q' : 2,
    'Master' : 0, 'Miss' : 1, 'Mrs' : 2, 'Mr' : 3, 'Others' : 4
})

In [14]:
#前処理が完了した訓練セット
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,GroupSize
0,0,3,0,22.0,7.2500,0,3,2
1,1,1,1,38.0,71.2833,1,2,2
2,1,3,1,26.0,7.9250,0,1,1
3,1,1,1,35.0,53.1000,0,2,2
4,0,3,0,35.0,8.0500,0,3,1


# モデル構築  

特にパラメータも指定せずに、訓練させてみる。

In [15]:
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
# モデルの設定
models = []
models.append(("SVM", SVC()))
models.append(("Random Forest Classifier", RandomForestClassifier()))
models.append(("Logistic Regression", LogisticRegression()))

モデルの評価方法としては、交差検証法を用い5分割交差検証を行う。

In [17]:
# 5 Fold Cross Validation
kf = KFold(n_splits=5) 
names = []
score = []
for name, model in models:
    names.append(name)
    scores = []
    for train_split, test_split in kf.split(train):
        train_data, test_data = train.iloc[train_split].iloc[:,1:], train.iloc[test_split].iloc[:,1:]
        train_labels, test_labels = train.iloc[train_split].iloc[:,0], train.iloc[test_split].iloc[:,0]
        model.fit(train_data, train_labels)
        result = model.predict(test_data)
        acc = accuracy_score(result, test_labels)
        scores.append(acc)
    score.append(np.mean(scores))

In [18]:
for i in range(len(names)):
    print("{}:""{}".format(names[i], score[i]))

SVM:0.711612579248007
Random Forest Classifier:0.8013495700207145
Logistic Regression:0.8024480572468772


# 結果(デフォルト)  
モデルのパラメータをいじらず、デフォルトで学習させた結果  
サポートベクターマシンで71.2%, ランダムフォレストで80.2%, ロジスティック回帰で80.2%の精度となった。  
  
さらに精度を上げるために、Grid_Serchで最適なパラメータを探してみる。

# パラメータの最適化

In [429]:
from sklearn.grid_search import GridSearchCV

/Users/sudatakamune/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## パラメータの組み合わせ

In [430]:
svm_params = [
    {
     'C' : [1, 10, 100, 1000], 
     'kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
     'degree': [2, 3, 4], 
     'gamma': [0.1, 0.01, 0.001, 0.0001]
    }
]
RandomForest_params = [
    {
     'n_estimators' : [200],
     'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
     'max_depth' : [5]
    }
]
LogisticRegression_params = [
    {'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2']
    }
]

In [431]:
train_data, test_data, train_labels, test_labels = train_test_split(
    train.iloc[:, 1:], 
    train["Survived"], 
    test_size=0.2, 
    random_state=0
)

In [ ]:
### svmのパラメータ
score = 'f1'
clf_svm = GridSearchCV(
    SVC(), 
    svm_params,
    cv=5, 
    scoring='%s_weighted' % score,
    n_jobs = -1
) 
clf_svm.fit(train_data, train_labels) 
clf_svm.best_params_

SVMの最適なパラメータは  
C=1000, gamma = 0.0001, degree = 2, kernel = 'rbf'  
であることがわかった。

In [432]:
# RandomForestのパラメータ
score = 'f1'
clf_RandomForest = GridSearchCV(
    RandomForestClassifier(), 
    RandomForest_params, 
    cv=5, 
    scoring='%s_weighted' % score,
    n_jobs = -1
) 
clf_RandomForest.fit(train_data, train_labels) 
clf_RandomForest.best_params_

{'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200}

RandomForestClassifierの最適なパラメータは  
n_estimators = 200, max_depth = 5, min_samples_split =5  
であることがわかった。

In [459]:
# LogisticRegressionのパラメータ
score = 'f1'
clf_LogisticRegression = GridSearchCV(
    LogisticRegression(), 
    LogisticRegression_params, 
    cv=5, 
    scoring='%s_weighted' % score,
    n_jobs = -1
) 
clf_LogisticRegression.fit(train_data, train_labels) 
clf_LogisticRegression.best_params_

{'C': 1000, 'penalty': 'l1'}

LogisticRegressionの最適なパラメータは  
C = 1000, penalty = 'l1'  
であることがわかった。

In [19]:
# モデルの設定
models = []
models.append(("SVM", SVC(C=1000, gamma = 0.0001, degree = 2, kernel = 'rbf')))
models.append(("Random Forest Classifier", RandomForestClassifier(n_estimators = 200, max_depth = 5, min_samples_split =10)))
models.append(("Logistic Regression", LogisticRegression(C = 1000, penalty='l1')))

In [20]:
# 5 Fold Cross Validation
kf = KFold(n_splits=5) 
names = []
score = []
for name, model in models:
    names.append(name)
    scores = []
    for train_split, test_split in kf.split(train):
        train_data, test_data = train.iloc[train_split].iloc[:,1:], train.iloc[test_split].iloc[:,1:]
        train_labels, test_labels = train.iloc[train_split].iloc[:,0], train.iloc[test_split].iloc[:,0]
        model.fit(train_data, train_labels)
        result = model.predict(test_data)
        acc = accuracy_score(result, test_labels)
        scores.append(acc)
    score.append(np.mean(scores))

In [21]:
for i in range(len(names)):
    print("{}:""{}".format(names[i], score[i]))

SVM:0.8092021844203126
Random Forest Classifier:0.8305191136777352
Logistic Regression:0.8047015253279769


モデルのパラメータのチューニングした結果  
サポートベクターマシンで80.9%, ランダムフォレストで83.4%, ロジスティック回帰で80.5%の精度となった。  

# kaggleに投稿  
同封されているテストデータから生存を予測し、kaggleに投稿してみる。

## テストデータの前処理  
訓練データと同じ前処理を行う。  
内容については省略する。

In [22]:
# テストデータ
test = pd.read_csv('./dataset/test.csv')

In [23]:
# 処理前のテストデータ
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [24]:
# それぞれの欠損値の数
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

訓練データと違う点は、テストデータは運賃(Fare)に欠損値があること

In [25]:
# 名前(Name)から敬称(Title)を抽出
test['Title'] = test['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [26]:
# 年齢に欠損のある敬称
Counter(test[test['Age'].isnull()].Title)

Counter({'Master': 4, 'Miss': 14, 'Mr': 57, 'Mrs': 10, 'Ms': 1})

In [27]:
# 敬称の種類と数
Counter(test['Title'])

Counter({'Col': 2,
         'Dona': 1,
         'Dr': 1,
         'Master': 21,
         'Miss': 78,
         'Mr': 240,
         'Mrs': 72,
         'Ms': 1,
         'Rev': 2})

テストセットの数は訓練セットの半分ほどなので、年齢の欠損値には訓練セットのものを使用する。  
"Ms."については、テストデータに1人しかいない上に訓練データにも1人しかいない。したがって訓練データの1人の年齢を入れる。

In [28]:
# 同敬称の年齢の平均値を補填
test.loc[(test["Title"].values == "Master") & (test["Age"].isnull()), "Age"] = Master_age
test.loc[(test["Title"].values == "Miss") & (test["Age"].isnull()), "Age"] = Miss_age
test.loc[(test["Title"].values == "Mr") & (test["Age"].isnull()), "Age"] = Mr_age
test.loc[(test["Title"].values == "Mrs") & (test["Age"].isnull()), "Age"] = Mrs_age
test.loc[(test["Title"].values == "Ms") & (test["Age"].isnull()), "Age"] = Ms_age

In [29]:
# 近い敬称をまとめる
test['Title'] = test['Title'].replace(['Dr', 'Rev', 'Col', 'Dona'], 'Others')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mlle', 'Miss')

In [30]:
# 特徴「家族の人数」を追加
test['GroupSize'] = test['SibSp'] + test['Parch'] + 1

In [31]:
# 補填と特徴を追加した訓練データ
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,GroupSize
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Mr,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Mrs,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Mr,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Mr,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Mrs,3


In [31]:
test[test['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,GroupSize
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,Mr,1


運賃に関連があると考えられる特徴としては、階級、グループ人数、寄港地だと考えられる。  
運賃に欠損値のある人物の特徴は「階級:3, グループ人数:1, 寄港地:S」であることから、同じ特徴の運賃の平均値を補填する。

In [32]:
#階級:3, グループ人数:1, 寄港地:Sの人の平均運賃
Fare_ave = test.query('Pclass == 3 & GroupSize == 1 & Embarked =="S"' ).Fare.mean()

In [33]:
# 運賃の補填
test = test.fillna({'Fare' : Fare_ave})

In [34]:
# 特徴の欠落と数値化
test= test.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)
test = test.replace({
    'male' : 0, 'female' : 1, 
    'S' : 0, 'C' : 1, 'Q' : 2,
    'Master' : 0, 'Miss' : 1, 'Mrs' : 2, 'Mr' : 3, 'Others' : 4
})

In [35]:
# 処理済みテストデータ
test.head()

,Pclass,Sex,Age,Fare,Embarked,Title,GroupSize
0,3,0,34.5,7.8292,2,3,1
1,3,1,47.0,7.0000,0,2,2
2,2,0,62.0,9.6875,2,3,1
3,3,0,27.0,8.6625,0,3,1
4,3,1,22.0,12.2875,0,2,3


In [36]:
# データの分割
train_data, train_labels = train.iloc[:,1:], train["Survived"]
test_data = test

## モデルの訓練と予測  
使用するモデルは、交差検証で一番いい精度を挙げたRandomForestClassifierを使用する。  

In [37]:
# モデルの訓練
RFC = RandomForestClassifier(n_estimators = 200, max_depth = 5, min_samples_split =5)
RFC.fit(train_data, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
# テストデータの予測
result = RFC.predict(test_data)

# kaggleへ提出  
テストデータを再び読み込み、予測結果を結合する。  
乗客IDと生存予測だけをcsvとして書き出し、kaggleに提出する。

In [456]:
test = pd.read_csv('./dataset/test.csv')
test['Survived'] = result

In [458]:
test[["PassengerId", "Survived"]].to_csv('./result/result_rfc.csv', index=False)

<img src="https://i.imgur.com/YRKDBI4.png">

## 結果  
提出結果は79.4%となった。  

交差検証で一番精度の良かったRandom Forest Classifierを用いたが、SVMとLogistic Regressionもやってみた。

In [39]:
SVM = SVC(C=1000, gamma = 0.0001, degree = 2, kernel = 'rbf')
SVM.fit(train_data, train_labels)
result_svm = SVM.predict(test_data)
test = pd.read_csv('./dataset/test.csv')
test['Survived'] = result_svm
test[["PassengerId", "Survived"]].to_csv('./result/result_svm.csv', index=False)

LR = LogisticRegression(C = 1000, penalty='l1')
LR.fit(train_data, train_labels)
result_lr = LR.predict(test_data)
test = pd.read_csv('./dataset/test.csv')
test['Survived'] = result_lr
test[["PassengerId", "Survived"]].to_csv('./result/result_lr.csv', index=False)

<img src="https://i.imgur.com/yd2CVZz.png">
<img src="https://i.imgur.com/IIrw145.png">

二つのモデルの予測結果は  
svm : 78.0%, Logistic Regression : 75.6%  
となった。  
交差検証の精度の順位と相違があったものの、RandomForestClassifierよりも良くなることはなかった。

# 考察  
さらに精度を上げるために  
・採用する特徴を選ぶとき、なんとなく関連ありそうなものを選んだ。  
　なんとなくで選ぶのではなく、ヒートマップなどで関連性を視覚化すべきであった。